Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import os


<li> Setting arbitrary passing_score = 70 for analysis going forward </li>
<li> Create data frames reading Schools data file </li>
 <li>   Rename the name column from 'name' to 'school_name' </li>
    

In [2]:
passing_score = 70
sch_csv_path = os.path.join("raw_data","schools_complete.csv")
schools_df = pd.read_csv(sch_csv_path)
# schools_df = schools_df.loc[schools_df["type"]=="District"]
schools_df = schools_df.reset_index(drop=True)
schools_df = schools_df.rename(columns={"name":"school_name"})
# schools_df


Compute number of schools into schools_count

In [3]:
schools_count = schools_df["school_name"].count()
# schools_count

Import students data file into students_df data frame
Rename school column as school_name

In [4]:
stu_csv_path = os.path.join("raw_data","students_complete.csv")
students_df = pd.read_csv(stu_csv_path)
students_df = students_df.rename(columns={"school":"school_name"})
# students_df.head()
# students_df.describe()

Merge school data frame with type='District' and students data

In [5]:
district_df = pd.merge(schools_df[schools_df.type == "District"],students_df,on="school_name")
# district_df.shape

Compute the number of schools into 'district_count'

In [6]:
district_count = len(district_df["school_name"].unique())
district_count

7

In [7]:
budget_total = district_df["budget"].sum()
# budget_total

In [8]:
avg_math_score = district_df["math_score"].mean()
# avg_math_score

In [9]:
avg_reading_score = district_df["reading_score"].mean()
# avg_reading_score

In [10]:
students_count = district_df["Student ID"].count()
# students_count

In [11]:
passing_math = district_df.loc[district_df["math_score"] >= passing_score].count() * 100/students_count
passing_math = passing_math["name"]

In [12]:
passing_reading = district_df.loc[district_df["reading_score"] >= passing_score].count() * 100/students_count
passing_reading = passing_reading["name"]

In [13]:
passing_list = [passing_math,passing_reading]
passing_rate = np.mean(passing_list)
# passing_rate

District Summary

Create a high level snapshot (in table form) of the district's key metrics, including:

Total Schools
Total Students
Total Budget
Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)

In [14]:
dict = {"Total Schools": district_count, "Total Students": students_count, "Total Budget": budget_total,
       "Average Math Score": avg_math_score, "Average Reading Score": avg_reading_score,
       "% Passing Math": passing_math, "% Passing Reading": passing_reading, "% Overall Passing Rate": passing_rate}
df = pd.DataFrame([dict], columns=dict.keys())
df.reset_index(drop=True)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,7,26976,70439053973,76.987026,80.962485,66.518387,80.905249,73.711818


create a school and students data from including all the data -- both District and Charter

In [15]:
schoolstu_df = pd.merge(schools_df,students_df,on="school_name")
school_sum = schoolstu_df.groupby(["school_name","type","name"])

In [16]:
# school_sum_no_students = district_df.groupby(["school_name"])
# school_sum_no_students['reading_score'].filter(lambda x: x > 70)

In [17]:
# t_school_math_sum = pd.DataFrame(district_df.groupby(["school_name"])["math_score"].mean().reset_index())
# t_school_math_sum = pd.DataFrame(district_df.groupby(["school_name"])(["math_score"]).mean().reset_index())
# t_school_math_sum


# pd.DataFrame(district_df.groupby(["school_name","grade"])["math_score"].mean().reset_index())

# t_school_math_pass = school_sum[school_sum["math_score"]> passing_score]

Compute the total of math_score greater than the passing score by each school.

In [18]:
school_sum_mp = school_sum.apply(lambda x: x[x["math_score"]>=passing_score])
# school_sum_mp["school_name"].value_counts()
school_sum_mp = pd.DataFrame(school_sum_mp["school_name"].value_counts().reset_index())
school_sum_mp.columns = ["school_name", "math_pass"]
# school_sum_bySchool = pd.DataFrame(school_sum["math_score"].sum())
# school_sum_bySchool
# math = school_sum_bySchool.loc[school_sum_bySchool["math_score"]>=passing_score]
# math.value_count()

In [20]:
# school_sum_mp

Compute the total of math_score by each school

In [21]:
school_sum_sCounts = school_sum.apply(lambda x: x[x["math_score"]>=0])
school_sum_sCounts = pd.DataFrame(school_sum_sCounts["school_name"].value_counts().reset_index())
school_sum_sCounts.columns = ["school_name", "students_count"]
school_sum_sCounts

,school_name,students_count
0,Bailey High School,4976
1,Johnson High School,4761
2,Hernandez High School,4635
3,Rodriguez High School,3999
4,Figueroa High School,2949
5,Huang High School,2917
6,Ford High School,2739
7,Wilson High School,2283
8,Cabrera High School,1858
9,Wright High School,1800


Merge school summary table with total of math_score by school

In [22]:
school_sum_all = pd.merge(school_sum_sCounts,school_sum_mp,on="school_name")
# type(school_sum_all)

pandas.core.frame.DataFrame

Compute the total reading_score greater then the passing score by each school

In [24]:
school_sum_rp = school_sum.apply(lambda x: x[x["reading_score"]>=passing_score])
# school_sum_mp["school_name"].value_counts()
school_sum_rp = pd.DataFrame(school_sum_rp["school_name"].value_counts().reset_index())
school_sum_rp.columns = ["school_name", "reading_pass"]
school_sum_rp

,school_name,reading_pass
0,Bailey High School,4077
1,Johnson High School,3867
2,Hernandez High School,3748
3,Rodriguez High School,3208
4,Figueroa High School,2381
5,Huang High School,2372
6,Wilson High School,2204
7,Ford High School,2172
8,Cabrera High School,1803
9,Wright High School,1739


Merge school summary and reading_score totals for each school

In [25]:
school_sum_all = pd.merge(school_sum_all,school_sum_rp,on="school_name") # run only once
school_sum_all

,school_name,students_count,math_pass,reading_pass
0,Bailey High School,4976,3318,4077
1,Johnson High School,4761,3145,3867
2,Hernandez High School,4635,3094,3748
3,Rodriguez High School,3999,2654,3208
4,Figueroa High School,2949,1946,2381
5,Huang High School,2917,1916,2372
6,Ford High School,2739,1871,2172
7,Wilson High School,2283,2143,2204
8,Cabrera High School,1858,1749,1803
9,Wright High School,1800,1680,1739


Compute % Passing Math by school

In [26]:
school_sum_all["% math pass"] = school_sum_all["math_pass"]* 100/school_sum_all["students_count"]


Compute % Passing Reading by school

In [28]:
school_sum_all["% reading pass"] = school_sum_all["reading_pass"]* 100/school_sum_all["students_count"]



In [29]:
school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass
0,Bailey High School,4976,3318,4077,66.680064,81.933280
1,Johnson High School,4761,3145,3867,66.057551,81.222432
2,Hernandez High School,4635,3094,3748,66.752967,80.862999
3,Rodriguez High School,3999,2654,3208,66.366592,80.220055
4,Figueroa High School,2949,1946,2381,65.988471,80.739234
5,Huang High School,2917,1916,2372,65.683922,81.316421
6,Ford High School,2739,1871,2172,68.309602,79.299014
7,Wilson High School,2283,2143,2204,93.867718,96.539641
8,Cabrera High School,1858,1749,1803,94.133477,97.039828
9,Wright High School,1800,1680,1739,93.333333,96.611111


In [30]:
school_sum_all = pd.merge(school_sum_all,schools_df[["school_name","budget","type"]],on="school_name") #run only once

Compute budget per student

In [31]:
school_sum_all["budget per student"] = school_sum_all["budget"]/school_sum_all["students_count"]
# school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student
0,Bailey High School,4976,3318,4077,66.680064,81.933280,3124928,District,628.0
1,Johnson High School,4761,3145,3867,66.057551,81.222432,3094650,District,650.0
2,Hernandez High School,4635,3094,3748,66.752967,80.862999,3022020,District,652.0
3,Rodriguez High School,3999,2654,3208,66.366592,80.220055,2547363,District,637.0
4,Figueroa High School,2949,1946,2381,65.988471,80.739234,1884411,District,639.0
5,Huang High School,2917,1916,2372,65.683922,81.316421,1910635,District,655.0
6,Ford High School,2739,1871,2172,68.309602,79.299014,1763916,District,644.0
7,Wilson High School,2283,2143,2204,93.867718,96.539641,1319574,Charter,578.0
8,Cabrera High School,1858,1749,1803,94.133477,97.039828,1081356,Charter,582.0
9,Wright High School,1800,1680,1739,93.333333,96.611111,1049400,Charter,583.0


In [32]:
school_sum_math = pd.DataFrame(schoolstu_df.groupby(["school_name"])["math_score"].sum().reset_index())
school_sum_math.columns = ["school_name", "math_total"]
# school_sum_math

,school_name,math_total
0,Bailey High School,383393
1,Cabrera High School,154329
2,Figueroa High School,226223
3,Ford High School,211184
4,Griffin High School,122360
5,Hernandez High School,358238
6,Holden High School,35784
7,Huang High School,223528
8,Johnson High School,366942
9,Pena High School,80654


In [33]:
school_sum_all = pd.merge(school_sum_all,school_sum_math,on="school_name")
# school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total
0,Bailey High School,4976,3318,4077,66.680064,81.933280,3124928,District,628.0,383393
1,Johnson High School,4761,3145,3867,66.057551,81.222432,3094650,District,650.0,366942
2,Hernandez High School,4635,3094,3748,66.752967,80.862999,3022020,District,652.0,358238
3,Rodriguez High School,3999,2654,3208,66.366592,80.220055,2547363,District,637.0,307294
4,Figueroa High School,2949,1946,2381,65.988471,80.739234,1884411,District,639.0,226223
5,Huang High School,2917,1916,2372,65.683922,81.316421,1910635,District,655.0,223528
6,Ford High School,2739,1871,2172,68.309602,79.299014,1763916,District,644.0,211184
7,Wilson High School,2283,2143,2204,93.867718,96.539641,1319574,Charter,578.0,190115
8,Cabrera High School,1858,1749,1803,94.133477,97.039828,1081356,Charter,582.0,154329
9,Wright High School,1800,1680,1739,93.333333,96.611111,1049400,Charter,583.0,150628


In [34]:
school_sum_all["Average Math Score"] = school_sum_all["math_total"]/school_sum_all["students_count"]
# school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score
0,Bailey High School,4976,3318,4077,66.680064,81.933280,3124928,District,628.0,383393,77.048432
1,Johnson High School,4761,3145,3867,66.057551,81.222432,3094650,District,650.0,366942,77.072464
2,Hernandez High School,4635,3094,3748,66.752967,80.862999,3022020,District,652.0,358238,77.289752
3,Rodriguez High School,3999,2654,3208,66.366592,80.220055,2547363,District,637.0,307294,76.842711
4,Figueroa High School,2949,1946,2381,65.988471,80.739234,1884411,District,639.0,226223,76.711767
5,Huang High School,2917,1916,2372,65.683922,81.316421,1910635,District,655.0,223528,76.629414
6,Ford High School,2739,1871,2172,68.309602,79.299014,1763916,District,644.0,211184,77.102592
7,Wilson High School,2283,2143,2204,93.867718,96.539641,1319574,Charter,578.0,190115,83.274201
8,Cabrera High School,1858,1749,1803,94.133477,97.039828,1081356,Charter,582.0,154329,83.061895
9,Wright High School,1800,1680,1739,93.333333,96.611111,1049400,Charter,583.0,150628,83.682222


In [35]:
school_sum_reading = pd.DataFrame(schoolstu_df.groupby(["school_name"])["reading_score"].sum().reset_index())
school_sum_reading.columns = ["school_name", "reading_total"]
# school_sum_reading

,school_name,reading_total
0,Bailey High School,403225
1,Cabrera High School,156027
2,Figueroa High School,239335
3,Ford High School,221164
4,Griffin High School,123043
5,Hernandez High School,375131
6,Holden High School,35789
7,Huang High School,236810
8,Johnson High School,385481
9,Pena High School,80851


In [36]:
school_sum_all = pd.merge(school_sum_all,school_sum_reading,on="school_name")
# school_sum_all

In [37]:
school_sum_all["Average Reading Score"] = school_sum_all["reading_total"]/school_sum_all["students_count"]
# school_sum_all

In [38]:
school_sum_all["Overall passing rate"] = (school_sum_all["% math pass"] + school_sum_all["% reading pass"])/2
# school_sum_all.sort(on="Overall passing rate")

In [39]:
school_sum_all = school_sum_all.sort_values("Overall passing rate",ascending=False).reset_index(drop=True)

School Summary


Create an overview table that summarizes key metrics about each school, including:


School Name
School Type
Total Students
Total School Budget
Per Student Budget
Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)

In [40]:
school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate
0,Cabrera High School,1858,1749,1803,94.133477,97.039828,1081356,Charter,582.0,154329,83.061895,156027,83.975780,95.586652
1,Thomas High School,1635,1525,1591,93.272171,97.308869,1043130,Charter,638.0,136389,83.418349,137093,83.848930,95.290520
2,Pena High School,962,910,923,94.594595,95.945946,585858,Charter,609.0,80654,83.839917,80851,84.044699,95.270270
3,Griffin High School,1468,1371,1426,93.392371,97.138965,917500,Charter,625.0,122360,83.351499,123043,83.816757,95.265668
4,Wilson High School,2283,2143,2204,93.867718,96.539641,1319574,Charter,578.0,190115,83.274201,191748,83.989488,95.203679
5,Wright High School,1800,1680,1739,93.333333,96.611111,1049400,Charter,583.0,150628,83.682222,151119,83.955000,94.972222
6,Shelton High School,1761,1653,1688,93.867121,95.854628,1056600,Charter,600.0,146796,83.359455,147441,83.725724,94.860875
7,Holden High School,427,395,411,92.505855,96.252927,248087,Charter,581.0,35784,83.803279,35789,83.814988,94.379391
8,Bailey High School,4976,3318,4077,66.680064,81.933280,3124928,District,628.0,383393,77.048432,403225,81.033963,74.306672
9,Hernandez High School,4635,3094,3748,66.752967,80.862999,3022020,District,652.0,358238,77.289752,375131,80.934412,73.807983


List the top 5 schools based on 'Overall passing rate'

In [41]:
school_sum_all.head(5)

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate
0,Cabrera High School,1858,1749,1803,94.133477,97.039828,1081356,Charter,582.0,154329,83.061895,156027,83.975780,95.586652
1,Thomas High School,1635,1525,1591,93.272171,97.308869,1043130,Charter,638.0,136389,83.418349,137093,83.848930,95.290520
2,Pena High School,962,910,923,94.594595,95.945946,585858,Charter,609.0,80654,83.839917,80851,84.044699,95.270270
3,Griffin High School,1468,1371,1426,93.392371,97.138965,917500,Charter,625.0,122360,83.351499,123043,83.816757,95.265668
4,Wilson High School,2283,2143,2204,93.867718,96.539641,1319574,Charter,578.0,190115,83.274201,191748,83.989488,95.203679


List the bottom 5 schools based on 'Overall passing rate'

In [42]:
school_sum_all = school_sum_all.sort_values("Overall passing rate",ascending=True).reset_index(drop=True)
school_sum_all.head(5)

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate
0,Rodriguez High School,3999,2654,3208,66.366592,80.220055,2547363,District,637.0,307294,76.842711,322898,80.744686,73.293323
1,Figueroa High School,2949,1946,2381,65.988471,80.739234,1884411,District,639.0,226223,76.711767,239335,81.158020,73.363852
2,Huang High School,2917,1916,2372,65.683922,81.316421,1910635,District,655.0,223528,76.629414,236810,81.182722,73.500171
3,Johnson High School,4761,3145,3867,66.057551,81.222432,3094650,District,650.0,366942,77.072464,385481,80.966394,73.639992
4,Ford High School,2739,1871,2172,68.309602,79.299014,1763916,District,644.0,211184,77.102592,221164,80.746258,73.804308


In [ ]:
Math Scores by Grade


Table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [43]:
school_grade_math_sum = pd.DataFrame(schoolstu_df.groupby(["school_name","grade"])["math_score"].mean().reset_index())
school_grade_math_sum.columns = ["school_name", "grade","math_score"]
school_grade_math_sum["dgrade"] = pd.to_numeric(school_grade_math_sum["grade"].str.extract('(^\d*)'))

school_grade_math_sum = school_grade_math_sum.sort_values(["school_name","dgrade"],ascending=True).reset_index(drop=True)
school_grade_math_sum = school_grade_math_sum.rename(columns={'math_score':'Average Math Score'})
school_grade_math_sum = school_grade_math_sum.drop(columns=["dgrade"])
school_grade_math_sum

/Users/rkagathi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


,school_name,grade,Average Math Score
0,Bailey High School,9th,77.083676
1,Bailey High School,10th,76.996772
2,Bailey High School,11th,77.515588
3,Bailey High School,12th,76.492218
4,Cabrera High School,9th,83.094697
5,Cabrera High School,10th,83.154506
6,Cabrera High School,11th,82.765560
7,Cabrera High School,12th,83.277487
8,Figueroa High School,9th,76.403037
9,Figueroa High School,10th,76.539974


Reading Scores by Grade


Table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [44]:
school_grade_reading_sum = pd.DataFrame(schoolstu_df.groupby(["school_name","grade"])["reading_score"].mean().reset_index())
school_grade_reading_sum.columns = ["school_name", "grade","reading_score"]
school_grade_reading_sum["dgrade"] = pd.to_numeric(school_grade_reading_sum["grade"].str.extract('(^\d*)'))

school_grade_reading_sum = school_grade_reading_sum.sort_values(["school_name","dgrade"],ascending=True).reset_index(drop=True)
school_grade_reading_sum = school_grade_reading_sum.rename(columns={'reading_score':'Average Reading Score'})
school_grade_reading_sum = school_grade_reading_sum.drop(columns=["dgrade"])
school_grade_reading_sum

/Users/rkagathi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


,school_name,grade,Average Reading Score
0,Bailey High School,9th,81.303155
1,Bailey High School,10th,80.907183
2,Bailey High School,11th,80.945643
3,Bailey High School,12th,80.912451
4,Cabrera High School,9th,83.676136
5,Cabrera High School,10th,84.253219
6,Cabrera High School,11th,83.788382
7,Cabrera High School,12th,84.287958
8,Figueroa High School,9th,81.198598
9,Figueroa High School,10th,81.408912


In [45]:
bins = [560,600,620,640,680]

# Create the names for the four bins
group_names = ['Low', 'Medium', 'High', 'Very High']

In [46]:
# Cut budget and place the scores into bins
pd.cut(school_sum_all["budget per student"], bins, labels=group_names)

0          High
1          High
2     Very High
3     Very High
4     Very High
5     Very High
6          High
7           Low
8           Low
9           Low
10          Low
11         High
12       Medium
13         High
14          Low
Name: budget per student, dtype: category
Categories (4, object): [Low < Medium < High < Very High]

In [47]:
school_sum_all["Budget bin"]= pd.cut(
    school_sum_all["budget per student"], bins, labels=group_names)
# school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate,Budget bin
0,Rodriguez High School,3999,2654,3208,66.366592,80.220055,2547363,District,637.0,307294,76.842711,322898,80.744686,73.293323,High
1,Figueroa High School,2949,1946,2381,65.988471,80.739234,1884411,District,639.0,226223,76.711767,239335,81.158020,73.363852,High
2,Huang High School,2917,1916,2372,65.683922,81.316421,1910635,District,655.0,223528,76.629414,236810,81.182722,73.500171,Very High
3,Johnson High School,4761,3145,3867,66.057551,81.222432,3094650,District,650.0,366942,77.072464,385481,80.966394,73.639992,Very High
4,Ford High School,2739,1871,2172,68.309602,79.299014,1763916,District,644.0,211184,77.102592,221164,80.746258,73.804308,Very High
5,Hernandez High School,4635,3094,3748,66.752967,80.862999,3022020,District,652.0,358238,77.289752,375131,80.934412,73.807983,Very High
6,Bailey High School,4976,3318,4077,66.680064,81.933280,3124928,District,628.0,383393,77.048432,403225,81.033963,74.306672,High
7,Holden High School,427,395,411,92.505855,96.252927,248087,Charter,581.0,35784,83.803279,35789,83.814988,94.379391,Low
8,Shelton High School,1761,1653,1688,93.867121,95.854628,1056600,Charter,600.0,146796,83.359455,147441,83.725724,94.860875,Low
9,Wright High School,1800,1680,1739,93.333333,96.611111,1049400,Charter,583.0,150628,83.682222,151119,83.955000,94.972222,Low


Scores by School Spending


Table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:


Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)

In [48]:
school_sum_all = school_sum_all.sort_values("Overall passing rate",ascending=False).reset_index(drop=True)
school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate,Budget bin
0,Cabrera High School,1858,1749,1803,94.133477,97.039828,1081356,Charter,582.0,154329,83.061895,156027,83.975780,95.586652,Low
1,Thomas High School,1635,1525,1591,93.272171,97.308869,1043130,Charter,638.0,136389,83.418349,137093,83.848930,95.290520,High
2,Pena High School,962,910,923,94.594595,95.945946,585858,Charter,609.0,80654,83.839917,80851,84.044699,95.270270,Medium
3,Griffin High School,1468,1371,1426,93.392371,97.138965,917500,Charter,625.0,122360,83.351499,123043,83.816757,95.265668,High
4,Wilson High School,2283,2143,2204,93.867718,96.539641,1319574,Charter,578.0,190115,83.274201,191748,83.989488,95.203679,Low
5,Wright High School,1800,1680,1739,93.333333,96.611111,1049400,Charter,583.0,150628,83.682222,151119,83.955000,94.972222,Low
6,Shelton High School,1761,1653,1688,93.867121,95.854628,1056600,Charter,600.0,146796,83.359455,147441,83.725724,94.860875,Low
7,Holden High School,427,395,411,92.505855,96.252927,248087,Charter,581.0,35784,83.803279,35789,83.814988,94.379391,Low
8,Bailey High School,4976,3318,4077,66.680064,81.933280,3124928,District,628.0,383393,77.048432,403225,81.033963,74.306672,High
9,Hernandez High School,4635,3094,3748,66.752967,80.862999,3022020,District,652.0,358238,77.289752,375131,80.934412,73.807983,Very High


In [49]:
bins = [0,2000,4000,6000]

# Create the names for the four bins
group_names = ['Small', 'Medium', 'Large']

In [50]:
# Cut school size and place the scores into bins
pd.cut(school_sum_all["students_count"], bins, labels=group_names)

0      Small
1      Small
2      Small
3      Small
4     Medium
5      Small
6      Small
7      Small
8      Large
9      Large
10    Medium
11     Large
12    Medium
13    Medium
14    Medium
Name: students_count, dtype: category
Categories (3, object): [Small < Medium < Large]

Scores by School Size


Repeat of the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

In [51]:
school_sum_all["School size bin"]= pd.cut(
    school_sum_all["students_count"], bins, labels=group_names)
school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate,Budget bin,School size bin
0,Cabrera High School,1858,1749,1803,94.133477,97.039828,1081356,Charter,582.0,154329,83.061895,156027,83.975780,95.586652,Low,Small
1,Thomas High School,1635,1525,1591,93.272171,97.308869,1043130,Charter,638.0,136389,83.418349,137093,83.848930,95.290520,High,Small
2,Pena High School,962,910,923,94.594595,95.945946,585858,Charter,609.0,80654,83.839917,80851,84.044699,95.270270,Medium,Small
3,Griffin High School,1468,1371,1426,93.392371,97.138965,917500,Charter,625.0,122360,83.351499,123043,83.816757,95.265668,High,Small
4,Wilson High School,2283,2143,2204,93.867718,96.539641,1319574,Charter,578.0,190115,83.274201,191748,83.989488,95.203679,Low,Medium
5,Wright High School,1800,1680,1739,93.333333,96.611111,1049400,Charter,583.0,150628,83.682222,151119,83.955000,94.972222,Low,Small
6,Shelton High School,1761,1653,1688,93.867121,95.854628,1056600,Charter,600.0,146796,83.359455,147441,83.725724,94.860875,Low,Small
7,Holden High School,427,395,411,92.505855,96.252927,248087,Charter,581.0,35784,83.803279,35789,83.814988,94.379391,Low,Small
8,Bailey High School,4976,3318,4077,66.680064,81.933280,3124928,District,628.0,383393,77.048432,403225,81.033963,74.306672,High,Large
9,Hernandez High School,4635,3094,3748,66.752967,80.862999,3022020,District,652.0,358238,77.289752,375131,80.934412,73.807983,Very High,Large


School Type, Budget and Overall Passing Rate summary

In [55]:
school_type_budget_sum = pd.DataFrame(school_sum_all.groupby(["type","Budget bin"])["Overall passing rate"].mean().reset_index())
school_type_budget_sum

,type,Budget bin,Overall passing rate
0,Charter,Low,95.000564
1,Charter,Medium,95.270270
2,Charter,High,95.278094
3,District,High,73.654616
4,District,Very High,73.688113


School Type, School Size and Overall Passing Rate summary

In [56]:
school_typ_size_sum= pd.DataFrame(school_sum_all.groupby(["type","School size bin"])["Overall passing rate"].mean().reset_index())
school_typ_size_sum

,type,School size bin,Overall passing rate
0,Charter,Small,95.089371
1,Charter,Medium,95.203679
2,District,Medium,73.490414
3,District,Large,73.918215


Three observable trends based on the data.
<li> Charter Schools fare well in Overall passing rate compared to District Schools </li>
<li> Smaller school size results in higher Overall passing rate </li>
<li> Budget per student is less in Charter Schools but still Overall passing rate is high compared to District Schools </li>